在运行dw相关表之前，首先运行下面部分以创建中间表

In [3]:
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import os
import sklearn
print(os.path.abspath(os.curdir))

/home/thd/人行征信_final


In [4]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",100)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()

In [15]:
# 定义udf 函数
def time_format(ts):
    try:
        res = ts[0:10].replace('.','')
        return res
    except:
        return 'None'
    else:
        return 'None'    
spark.udf.register('time_format',lambda x:time_format(x))

def format_year_month(time):
    try:
        res = time[0:7].replace('.','-')
        return res
    except:
        return 'None'
spark.udf.register('format_year_month',lambda x:format_year_month(x))

    
def get_time(ts):
    try:
        res = ts.replace('.','-')
        return res
    except:
        return 'None'
    else:
        return 'None'


spark.udf.register('get_time',lambda x:get_time(x))
# time_udfs = udf(get_time, StringType())


def timestampdiff(flag,t1,t2):
    try:
        date1 = t1[0:10].replace('.','-')
        date2 = t2[0:10].replace('.','-')
        year = (int(date1[0:4])-int(date2[0:4]))
        month = (int(date1[5:7])-int(date2[5:7]))
        day = int(date1[8:10])-int(date2[8:10])
        if(flag == 'year'):
            if(month<0 or (month==0 and day<0)):
                result = year -1
                return result
            return year
        if(flag == 'month'):
            res = year*12+month+int((day-30)/31)
            return res
    
    except:
        return 'None'
    
                                                                                      
spark.udf.register('timestampdiff',lambda x1,x2,x3:timestampdiff(x1,x2,x3))
# timestampdiff = udf(get_time, StringType())   


def get_city(address):
    try:
        province_loc = address.find("省")
        city_loc = address.find("市")
        if(province_loc < 0 or (province_loc>=0 and province_loc>=city_loc) ): #江西赣州市兴国县江西省赣州市兴国县潋江镇罗廖村26号
            city = address[0:city_loc+1]
        if(province_loc>=0 and province_loc<city_loc):
            city = address[province_loc+1:city_loc+1]
        return city
    except:
        return 'None'



spark.udf.register('get_city',lambda x:get_city(x))
# time_udfs = udf(get_time, StringType())
                                                                                    

In [18]:
#创建DW.dw_baseinfo_detail
table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRMESSAGEHEADER") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table1.createOrReplaceTempView("ICRMESSAGEHEADER")

table2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCREDITCUE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table2.createOrReplaceTempView("ICRCREDITCUE")

table3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLOANCARDINFO") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table3.createOrReplaceTempView("ICRLOANCARDINFO")

table4 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCREDITCUE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table4.createOrReplaceTempView("ICRCREDITCUE")

table5 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST2YEAROVERDUECARD") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table5.createOrReplaceTempView("ICRLATEST2YEAROVERDUECARD")

table6 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST5YEAROVERDUEDETAIL") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table6.createOrReplaceTempView("ICRLATEST5YEAROVERDUEDETAIL")

table7 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST2YEAROVERDUE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table7.createOrReplaceTempView("ICRLATEST2YEAROVERDUE")

table8 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLOANINFO") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table8.createOrReplaceTempView("ICRLOANINFO")

table9 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRUNDESTORYLOANCARD") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table9.createOrReplaceTempView("ICRUNDESTORYLOANCARD")

table10 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRUNDESTORYSTANDARDLOANCARD") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table10.createOrReplaceTempView("ICRUNDESTORYSTANDARDLOANCARD")

table11 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRUNPAIDLOAN") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table11.createOrReplaceTempView("ICRUNPAIDLOAN")


table12 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICROVERDUESUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table12.createOrReplaceTempView("ICROVERDUESUMMARY")


table13 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRFELLBACKSUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table13.createOrReplaceTempView("ICRFELLBACKSUMMARY")


table14 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRGUARANTEE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table14.createOrReplaceTempView("ICRGUARANTEE")


table15 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRGUARANTEESUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table15.createOrReplaceTempView("ICRGUARANTEESUMMARY")


#######

table16 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCIVILJUDGEMENT") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table16.createOrReplaceTempView("ICRCIVILJUDGEMENT")



table17 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRFORCEEXECUTION") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table17.createOrReplaceTempView("ICRFORCEEXECUTION")


table18 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRADMINPUNISHMENT") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table18.createOrReplaceTempView("ICRADMINPUNISHMENT")


table19 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRTAXARREAR") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table19.createOrReplaceTempView("ICRTAXARREAR")


table20 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRACCFUND") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table20.createOrReplaceTempView("ICRACCFUND")



table21 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRENDOWMENTINSURANCEDEPOSIT") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table21.createOrReplaceTempView("ICRENDOWMENTINSURANCEDEPOSIT")


table22 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRRECORDDETAIL") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table22.createOrReplaceTempView("ICRRECORDDETAIL")



table23 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRRECORDDETAIL") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table23.createOrReplaceTempView("ICRRECORDDETAIL")


table24 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.QUERY_HISTORY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table24.createOrReplaceTempView("QUERY_HISTORY")

table25 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRRECORDSUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table25.createOrReplaceTempView("ICRRECORDSUMMARY")


###########

spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='准贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_standcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_creditcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		ordue.LASTMONTHS,ordue.AMOUNT,loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'  and BIZTYPE='贷记卡'
""").createOrReplaceTempView("temp_creditcard_overdue")


spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		format_year_month(concat(month,'.01')),
        gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_overdue
""").createOrReplaceTempView("temp_creditcard_5yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,
		STATE,
		FINANCEORG
from ICRLATEST2YEAROVERDUE ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_loan_2yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		ordue.LASTMONTHS,
		ordue.AMOUNT,
		loanfo.GUARANTEETYPE,
		STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'
""").createOrReplaceTempView("temp_loan_overdue")




spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
        MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		format_year_month(concat(month,'.01')),gap_months,LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_overdue
""").createOrReplaceTempView("temp_loan_5yearoverdue")




In [17]:
spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='准贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_standcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_creditcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		ordue.LASTMONTHS,ordue.AMOUNT,loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'  and BIZTYPE='贷记卡'
""").createOrReplaceTempView("temp_creditcard_overdue")


spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		format_year_month(concat(month,'.01')),
        gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_overdue
""").createOrReplaceTempView("temp_creditcard_5yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,
		STATE,
		FINANCEORG
from ICRLATEST2YEAROVERDUE ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_loan_2yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		ordue.LASTMONTHS,
		ordue.AMOUNT,
		loanfo.GUARANTEETYPE,
		STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'
""").createOrReplaceTempView("temp_loan_overdue")




spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
        MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		format_year_month(concat(month,'.01')),gap_months,LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_overdue
""").createOrReplaceTempView("temp_loan_5yearoverdue")




中间表已运行完成，spark sql 创建所需要的表